In [1]:
import pandas as pd

scoreboards = pd.read_csv('datasets/esports.csv')
scoreboards

,Battletag,Eliminations,Deaths,Assists,Damage,Healing,Mitigated,Result
0,JunBin,22,5,10,14600,2421,7580,Defeat
1,HeeSang,20,7,6,8693,138,58,Defeat
2,LIP,15,6,0,10286,0,0,Defeat
3,CH0R0NG,14,4,16,6098,10225,0,Defeat
4,shu,14,8,9,6171,6811,696,Defeat
...,...,...,...,...,...,...,...,...
705,HanBin,24,7,11,11047,1493,34240,Defeat
706,Proper,29,12,3,13367,771,3802,Defeat
707,Stalk3r,24,9,0,16004,825,871,Defeat
708,ChiYo,16,9,18,6899,12420,3900,Defeat


In [2]:
scoreboards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Battletag     710 non-null    object
 1   Eliminations  710 non-null    int64 
 2   Deaths        710 non-null    int64 
 3   Assists       710 non-null    int64 
 4   Damage        710 non-null    int64 
 5   Healing       710 non-null    int64 
 6   Mitigated     710 non-null    int64 
 7   Result        710 non-null    object
dtypes: int64(6), object(2)
memory usage: 44.5+ KB


In [3]:
players = 10
scoreboards.set_index(
    pd.Index((i, j) for i in range(scoreboards.index.stop // players) for j in range(players)), inplace=True
)
scoreboards

Battletag  Eliminations  Deaths  Assists  Damage  Healing  Mitigated  \
0  0    JunBin            22       5       10   14600     2421       7580   
   1   HeeSang            20       7        6    8693      138         58   
   2       LIP            15       6        0   10286        0          0   
   3   CH0R0NG            14       4       16    6098    10225          0   
   4       shu            14       8        9    6171     6811        696   
...        ...           ...     ...      ...     ...      ...        ...   
70 5    HanBin            24       7       11   11047     1493      34240   
   6    Proper            29      12        3   13367      771       3802   
   7   Stalk3r            24       9        0   16004      825        871   
   8     ChiYo            16       9       18    6899    12420       3900   
   9   Fielder            16       8       24    7738    21655        592   

      Result  
0  0  Defeat  
   1  Defeat  
   2  Defeat  
   3  Defeat  
   4  Defeat  
...      ...  
70 5  Defeat  
   6  Defeat  
   7  Defeat  
   8  Defeat  
   9  Defeat  

[710 rows x 8 columns]

In [4]:
from sklearn import utils

scoreboards = scoreboards.loc[utils.shuffle(range(len(scoreboards.index) // players))]
scoreboards

Battletag  Eliminations  Deaths  Assists  Damage  Healing  Mitigated  \
6  0    DONGHAK            31       5        8   10800      400       7318   
   1      Proud            36       7        0   10437        0          0   
   2       ZEST            27       6        2   10081      337       1021   
   3     skewed            25       4       21    7115     9067       1506   
   4  vigilante            20       9       19    6057     9982        535   
...         ...           ...     ...      ...     ...      ...        ...   
32 5     Kellan             5       6        1    5864        0       6373   
   6     seeker             4       8        0    4241      956          0   
   7  sugarfree             8       3        0    6929        0          0   
   8      Bliss             5       4        4    3473     4820       1127   
   9       cjay             2       4        4    3551     5569          0   

       Result  
6  0  Victory  
   1  Victory  
   2  Victory  
   3  Victory  
   4  Victory  
...       ...  
32 5   Defeat  
   6   Defeat  
   7   Defeat  
   8   Defeat  
   9   Defeat  

[710 rows x 8 columns]

In [5]:
X = scoreboards.drop(['Battletag', 'Result'], axis=1)
y = scoreboards['Result'][::players].copy()

In [6]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler().fit(X)
X = pd.DataFrame(scaler.transform(X), X.index, X.columns)
X

Eliminations    Deaths   Assists    Damage   Healing  Mitigated
6  0      0.596154  0.294118  0.190476  0.375000  0.016487   0.206089
   1      0.692308  0.411765  0.000000  0.361145  0.000000   0.000000
   2      0.519231  0.352941  0.047619  0.347557  0.013891   0.028753
   3      0.480769  0.235294  0.500000  0.234351  0.373727   0.042412
   4      0.384615  0.529412  0.452381  0.193969  0.411442   0.015067
...            ...       ...       ...       ...       ...        ...
32 5      0.096154  0.352941  0.023810  0.186603  0.000000   0.179476
   6      0.076923  0.470588  0.000000  0.124656  0.039405   0.000000
   7      0.153846  0.176471  0.000000  0.227252  0.000000   0.000000
   8      0.096154  0.235294  0.095238  0.095344  0.198673   0.031738
   9      0.038462  0.235294  0.095238  0.098321  0.229545   0.000000

[710 rows x 6 columns]

In [7]:
features = X.shape[-1]
total_features = players * features
X = X.values.reshape(-1, total_features)
print(X.shape)
print(y.shape)

(71, 60)
(71,)


In [8]:
from sklearn import model_selection, neighbors

radius = 1.2
model_selection.cross_val_score(neighbors.RadiusNeighborsClassifier(radius=radius), X, y).mean()

np.float64(0.8466666666666667)

In [9]:
from sklearn import neural_network

classifier_iter = 700
model_selection.cross_val_score(neural_network.MLPClassifier(max_iter=classifier_iter), X, y).mean()

np.float64(0.8142857142857143)

In [10]:
from sklearn import ensemble, linear_model, svm, tree, model_selection
import operator

estimators = (
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(n_jobs=-1),
    ensemble.ExtraTreesClassifier(n_jobs=-1),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(n_jobs=-1),
    ensemble.HistGradientBoostingClassifier(),
    linear_model.LogisticRegression(n_jobs=-1),
    linear_model.LogisticRegressionCV(n_jobs=-1),
    linear_model.PassiveAggressiveClassifier(n_jobs=-1),
    linear_model.Perceptron(n_jobs=-1),
    linear_model.RidgeClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(n_jobs=-1),
    neighbors.KNeighborsClassifier(n_jobs=-1),
    neighbors.RadiusNeighborsClassifier(radius, n_jobs=-1),
    neighbors.NearestCentroid(),
    neural_network.MLPClassifier(max_iter=classifier_iter),
    svm.LinearSVC(dual='auto'),
    svm.NuSVC(),
    svm.SVC(),
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier()
)
scores = [(estimator, model_selection.cross_val_score(estimator, X, y, n_jobs=-1).mean()) for estimator in estimators]
scores.sort(key=operator.itemgetter(1), reverse=True)
scores

[(LogisticRegression(n_jobs=-1), np.float64(0.900952380952381)),
 (LogisticRegressionCV(n_jobs=-1), np.float64(0.8723809523809523)),
 (KNeighborsClassifier(n_jobs=-1), np.float64(0.8723809523809523)),
 (NuSVC(), np.float64(0.8723809523809523)),
 (SVC(), np.float64(0.8723809523809523)),
 (RadiusNeighborsClassifier(n_jobs=-1, radius=1.2),
  np.float64(0.8466666666666667)),
 (HistGradientBoostingClassifier(), np.float64(0.8428571428571429)),
 (RidgeClassifier(), np.float64(0.8428571428571429)),
 (RidgeClassifierCV(), np.float64(0.8428571428571429)),
 (Perceptron(n_jobs=-1), np.float64(0.8295238095238094)),
 (RandomForestClassifier(n_jobs=-1), np.float64(0.8285714285714286)),
 (MLPClassifier(max_iter=700), np.float64(0.8285714285714286)),
 (NearestCentroid(), np.float64(0.8285714285714285)),
 (LinearSVC(), np.float64(0.8152380952380952)),
 (ExtraTreesClassifier(n_jobs=-1), np.float64(0.8142857142857143)),
 (AdaBoostClassifier(), np.float64(0.7999999999999999)),
 (DecisionTreeClassifier(), 

In [11]:
for estimator, _ in scores:
    estimator = estimator.fit(X, y)
    if hasattr(estimator, 'feature_importances_'):
        importances = estimator.feature_importances_.reshape(-1, features)
        tanks, damages, supports = [0, 5], [1, 2, 6, 7], [3, 4, 8, 9]
        importances_tanks = importances[tanks].sum(0)
        importances_damages = importances[damages].sum(0)
        importances_supports = importances[supports].sum(0)
        importances[tanks] = importances_tanks / 2
        importances[damages] = importances_damages / 4
        importances[supports] = importances_supports / 4

        titles = '처치', '죽음', '도움', '피해', '치유', '경감'

        print('**특성 중요도**')
        for importance, title in sorted(zip(importances.sum(0), titles), reverse=True):
            print(f'{title} (10인): {importance * 100:.2f}%')

        print('\n**역할 중요도**')
        for importance, title in sorted(
                zip((importances[tanks].sum(), importances[damages].sum(), importances[supports].sum()), ('돌격 (2인)', '공격 (4인)', '지원 (4인)')),
                reverse=True
        ):
            print(f'{title}: {importance * 100:.2f}%')

        print('\n**돌격 특성 중요도**')
        for importance, title in sorted(zip(importances_tanks, titles), reverse=True):
            print(f'{title} (2인): {importance * 100:.2f}%')

        print('\n**공격 특성 중요도**')
        for importance, title in sorted(zip(importances_damages, titles), reverse=True):
            print(f'{title} (4인): {importance * 100:.2f}%')

        print('\n**지원 특성 중요도**')
        for importance, title in sorted(zip(importances_supports, titles), reverse=True):
            print(f'{title} (4인): {importance * 100:.2f}%')

        break

**특성 중요도**
처치 (10인): 31.72%
죽음 (10인): 29.70%
도움 (10인): 15.29%
피해 (10인): 10.09%
치유 (10인): 6.78%
경감 (10인): 6.41%

**역할 중요도**
지원 (4인): 41.34%
공격 (4인): 29.85%
돌격 (2인): 28.81%

**돌격 특성 중요도**
죽음 (2인): 15.01%
처치 (2인): 6.87%
피해 (2인): 2.14%
도움 (2인): 1.73%
경감 (2인): 1.61%
치유 (2인): 1.46%

**공격 특성 중요도**
처치 (4인): 13.82%
죽음 (4인): 7.83%
피해 (4인): 3.94%
치유 (4인): 1.81%
경감 (4인): 1.55%
도움 (4인): 0.89%

**지원 특성 중요도**
도움 (4인): 12.68%
처치 (4인): 11.02%
죽음 (4인): 6.87%
피해 (4인): 4.01%
치유 (4인): 3.52%
경감 (4인): 3.25%


In [12]:
averages = importances.sum(0) / players

In [13]:
len_ = len(scoreboards.index)
for i in range(len_ - 10, len_ - 101, -players):
    scoreboards_i = scoreboards[i:i + players]
    print(scoreboards_i)
    contributions = scaler.transform(scoreboards_i.drop(['Battletag', 'Result'], axis=1))
    contributions[:, 1] = 1 - contributions[:, 1]
    contributions = (contributions * averages).sum(1)
    contributions *= players / contributions.sum()
    print(f'TEAM 1:', end='')
    for contribution in contributions[:5]:
        print(f' {contribution:.2f}', end='')
    print(f'\nTEAM 2:', end='')
    for contribution in contributions[5:]:
        print(f' {contribution:.2f}', end='')
    print('\n')

      Battletag  Eliminations  Deaths  Assists  Damage  Healing  Mitigated  \
32 0     JunBin            14       1        3    5824        0       4227   
   1    HeeSang            15       0        1    7085        0        370   
   2        LIP            19       0        0    7203        0          0   
   3    CH0R0NG            10       4       13    3128     5034       2201   
   4        shu             7       4        6    3306     4708        127   
   5     Kellan             5       6        1    5864        0       6373   
   6     seeker             4       8        0    4241      956          0   
   7  sugarfree             8       3        0    6929        0          0   
   8      Bliss             5       4        4    3473     4820       1127   
   9       cjay             2       4        4    3551     5569          0   

       Result  
32 0  Victory  
   1  Victory  
   2  Victory  
   3  Victory  
   4  Victory  
   5   Defeat  
   6   Defeat  
   7   Defeat

In [14]:
from tensorflow.keras import backend, layers
from tensorflow import keras

backend.clear_session()
model = keras.Sequential()
model.add(keras.Input((total_features,)))
model.add(layers.Dense(300, 'relu'))
model.add(layers.Dense(100, 'relu'))
model.add(layers.Dense(1, 'sigmoid'))
model.summary()

2025-05-04 05:10:21.981097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746335421.996527   18381 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746335422.001265   18381 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746335422.014035   18381 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746335422.014050   18381 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746335422.014051   18381 computation_placer.cc:177] computation placer alr

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 300)            │        18,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,501 (189.46 KB)

 Trainable params: 48,501 (189.46 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
from tensorflow.keras import optimizers, callbacks

model.compile('sgd', 'binary_crossentropy', metrics=['accuracy'])
transform = preprocessing.LabelEncoder().fit_transform(y)
stopping = callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    X,
    transform,
    epochs=256,
    callbacks=[stopping],
    validation_split=.25
)
history.history['val_loss'][-11], history.history['val_accuracy'][-11]

Epoch 1/256
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.3536 - loss: 0.7197 - val_accuracy: 0.4444 - val_loss: 0.7040
Epoch 2/256
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.3640 - loss: 0.7102 - val_accuracy: 0.4444 - val_loss: 0.6998
Epoch 3/256
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3536 - loss: 0.7042 - val_accuracy: 0.5000 - val_loss: 0.6962
Epoch 4/256
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3662 - loss: 0.6973 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 5/256
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5649 - loss: 0.6910 - val_accuracy: 0.5000 - val_loss: 0.6901
Epoch 6/256
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.7154 - loss: 0.6859 - val_accuracy: 0.6111 - val_loss: 0.6876
Epoch 7/256
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.7488 - loss: 0.6804 - val_accuracy: 0.6111 - val_loss: 0.6850
Epoch 8/256
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6568 - loss: 0.6775 - val_accuracy: 0.6667 - val_loss

(0.35339343547821045, 0.9444444179534912)